In [2]:
import os
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

class ImageSplicer:
    """Class to handle image splicing."""
    
    def __init__(self, file_path, n):
        self.file_path = file_path
        self.n = n

    def splice_image(self):
        """Splices the image into n x n tiles."""
        img = Image.open(self.file_path)
        width, height = img.size
        tile_width = width // self.n
        tile_height = height // self.n
        cropped_images = []

        for i in range(self.n):
            for j in range(self.n):
                left = j * tile_width
                upper = i * tile_height
                right = left + tile_width
                lower = upper + tile_height
                cropped_img = img.crop((left, upper, right, lower))
                cropped_images.append((cropped_img, i * self.n + j))  # Store image along with index

        return cropped_images

class CloudMasker:
    """Class to handle cloud masking and inpainting."""
    
    def __init__(self, t1=0.1, t2=0.1, kernel_size=10, inpaint_radius=5):
        self.t1 = t1
        self.t2 = t2
        self.kernel_size = kernel_size
        self.inpaint_radius = inpaint_radius

    def mask_clouds(self, image):
        """Masks the clouds in an image and inpaints the masked regions."""
        image = np.array(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        R = image[:, :, 2].astype(float)
        G = image[:, :, 1].astype(float)
        B = image[:, :, 0].astype(float)
        rg = R / G
        gb = G / B

        cloud_mask1 = np.logical_and(np.abs(rg - 1) < self.t1, np.abs(gb - 1) < self.t2).astype(np.uint8)
        cloud_mask = cv2.dilate(cloud_mask1, kernel=np.ones((self.kernel_size, self.kernel_size), dtype=np.uint8), iterations=1)
        
        R_inpaint = cv2.inpaint(R.astype(np.uint8), cloud_mask, self.inpaint_radius, cv2.INPAINT_TELEA)
        G_inpaint = cv2.inpaint(G.astype(np.uint8), cloud_mask, self.inpaint_radius, cv2.INPAINT_TELEA)
        B_inpaint = cv2.inpaint(B.astype(np.uint8), cloud_mask, self.inpaint_radius, cv2.INPAINT_TELEA)
        
        inpainted_image = np.stack([B_inpaint, G_inpaint, R_inpaint], axis=2)
        cloud_cover = np.sum(cloud_mask) / cloud_mask.size * 100

        # Print cloud cover percentage
        print(f"Cloud cover: {cloud_cover:.2f}%")

        return cloud_cover, Image.fromarray(inpainted_image)

class ImageProcessor:
    """Main class to process images."""
    
    def __init__(self, image_path, tflite_model_path, n=3, threshold=20):
        self.image_path = image_path
        self.tflite_model_path = tflite_model_path
        self.n = n
        self.threshold = threshold
        self.interpreter = self._load_tflite_model()

    def _load_tflite_model(self):
        """Loads the TensorFlow Lite model."""
        interpreter = tf.lite.Interpreter(model_path=self.tflite_model_path)
        interpreter.allocate_tensors()
        return interpreter

    def _predict_with_tflite(self, image):
        """Predicts the class of an image using the TensorFlow Lite model."""
        input_details = self.interpreter.get_input_details()[0]
        output_details = self.interpreter.get_output_details()[0]

        image = cv2.resize(np.array(image), (IMG_WIDTH, IMG_HEIGHT))
        image = np.expand_dims(image, axis=0).astype(np.float32)
        
        self.interpreter.set_tensor(input_details['index'], image)
        self.interpreter.invoke()
        predictions = self.interpreter.get_tensor(output_details['index'])
        return np.argmax(predictions)

    def process_images(self):
        """
        Processes the images by splicing, masking, and predicting.
        - Splices the image into tiles
        - Masks clouds and filters tiles based on cloud cover
        - Predicts the class of each filtered tile
        """
        start_time = time.time()  # Record start time
        
        splicer = ImageSplicer(self.image_path, self.n)
        images = splicer.splice_image()

        masker = CloudMasker()
        if not os.path.exists(FILTERED_IMAGE_FOLDER):
            os.makedirs(FILTERED_IMAGE_FOLDER)

        base_name = os.path.splitext(os.path.basename(self.image_path))[0]
        for cropped_img, idx in images:
            cloud_cover, inpainted_image = masker.mask_clouds(cropped_img)
            
            status = "Accepted" if cloud_cover < self.threshold else "Rejected"
            print(f"Part {idx} Cloud Coverage: {cloud_cover:.2f}% - {status}")
            
            if cloud_cover < self.threshold:
                # Resize for model processing
                downgraded_img = inpainted_image.resize((IMG_WIDTH, IMG_HEIGHT), Image.LANCZOS)
                img_array = np.array(downgraded_img) / 255.0
                img_array_exp = np.expand_dims(img_array, axis=0)
                prediction = self._predict_with_tflite(downgraded_img)
                
                predicted_class = prediction
                print(f"Predicted class index: {predicted_class}")

                # Check if the predicted class indicates vegetation (class index 1)
                if predicted_class == 1:
                    class_folder = FILTERED_IMAGE_FOLDER
                    os.makedirs(class_folder, exist_ok=True)
                    output_name = f"{base_name}_Q{idx}.png"
                    output_path = os.path.join(class_folder, output_name)
                    
                    # Save the original, high-resolution quadrant
                    cropped_img.save(output_path)
                    print(f"Original high-resolution quadrant saved to {output_path}")

                    # Plotting original, masked, and inpainted images
                    self._plot_images(cropped_img, inpainted_image, cloud_cover, predicted_class, idx)
        
        end_time = time.time()  # Record end time
        elapsed_time = end_time - start_time
        print(f"Time taken for processing: {elapsed_time:.2f} seconds")


FILTERED_IMAGE_FOLDER = r'C:\Users\abudh\Desktop\CropWatch\Filtered_Image'
TFLITE_MODEL_PATH = r'C:\Users\abudh\Desktop\CropWatch\resnet50_veg.tflite'
TILE_COUNT = 3
THRESHOLD_PERCENTAGE = 20
IMG_WIDTH = 64  # Define image width for model input
IMG_HEIGHT = 64  # Define image height for model input
TEST_FOLDER = r'C:\Users\abudh\Desktop\CropWatch\Test4'  # Add this configuration # Processing and visualizing images
start_time = time.time()  # Record start time for the entire processing

for image_file in os.listdir(TEST_FOLDER):
    image_path = os.path.join(TEST_FOLDER, image_file)
    image_processor = ImageProcessor(image_path, TFLITE_MODEL_PATH, n=TILE_COUNT, threshold=THRESHOLD_PERCENTAGE)
    image_processor.process_images()

end_time = time.time()  # Record end time for the entire processing
total_elapsed_time = end_time - start_time
print(f"Total time taken for all images: {total_elapsed_time:.2f} seconds")


Cloud cover: 98.27%
Part 0 Cloud Coverage: 98.27% - Rejected
Cloud cover: 99.10%
Part 1 Cloud Coverage: 99.10% - Rejected
Cloud cover: 99.94%
Part 2 Cloud Coverage: 99.94% - Rejected
Cloud cover: 99.82%
Part 3 Cloud Coverage: 99.82% - Rejected
Cloud cover: 100.00%
Part 4 Cloud Coverage: 100.00% - Rejected
Cloud cover: 99.99%
Part 5 Cloud Coverage: 99.99% - Rejected
Cloud cover: 99.99%
Part 6 Cloud Coverage: 99.99% - Rejected
Cloud cover: 100.00%
Part 7 Cloud Coverage: 100.00% - Rejected
Cloud cover: 99.98%
Part 8 Cloud Coverage: 99.98% - Rejected
Time taken for processing: 137.58 seconds
Cloud cover: 21.25%
Part 0 Cloud Coverage: 21.25% - Rejected
Cloud cover: 10.97%
Part 1 Cloud Coverage: 10.97% - Accepted
Predicted class index: 0
Cloud cover: 11.94%
Part 2 Cloud Coverage: 11.94% - Accepted
Predicted class index: 0
Cloud cover: 11.75%
Part 3 Cloud Coverage: 11.75% - Accepted
Predicted class index: 0
Cloud cover: 17.62%
Part 4 Cloud Coverage: 17.62% - Accepted
Predicted class index: 0
